In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [3]:
# 학습을 위한 데이터 x와 y에 값을 준다.
x_train = [[1., 2.],
          [2., 3.],
          [3., 1.],
          [4., 3.],
          [5., 3.],
          [6., 2.]]
y_train = [[0.],
          [0.],
          [0.],
          [1.],
          [1.],
          [1.]]

x_test = [[5.,2.]]
y_test = [[1.]]

# tf 데이터를 이용해 x 값과 y 값을 실제 x의 길이로 batch를 토대로 데이터 값을 가지고 온다..
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.zeros([2,1]), name='weight') # 2행1열 모양으로 w 값을 선언한다.
b = tf.Variable(tf.zeros([1]), name='bias')

In [4]:
def logistic_regression(features):
    hypothesis  = tf.div(1., 1. + tf.exp(tf.matmul(features, W) + b)) # w * x + b에 대한 linear값을 tf.exp 이용한다.
    return hypothesis

def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.log(logistic_regression(features)) + (1 - labels) * tf.log(1 - hypothesis))
    return cost # cost값을 구하기 위한 labels과  hypothesis은 features이 들어와 오류를 얻기 위해 hypothesis로 지정한다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [5]:
def accuracy_fn(hypothesis, labels): # 0과 1을 결정하게되는 모델을 hypothesis 값이 0.5를 통해서 예측 값이 나온다.
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) # 실제 값과 예측 값을 비교해서  accuracy로 출력한다.
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

In [6]:
def grad(hypothesis, features, labels): # 학습으로 hypothesis와 labels이 나오면, 
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features,labels) # hypothesis 값과 labels 값을 비교해 loss 값을 구한다.
    return tape.gradient(loss_value, [W,b]) # gradient를 통해서 모델 값을 바꿀 수 있다.

In [7]:
EPOCHS = 1001  # 1001 EPOCHS를 실행한다.

for step in range(EPOCHS): 
    for features, labels  in tfe.Iterator(dataset): # dataset을 토대로 tfe.Iterator를 이용한다.
        grads = grad(logistic_regression(features), features, labels) # features, labels 값들을 가설 함수에 집어넣어서 grad 값을 나오게한다.
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b])) # optimizer.apply_gradients 이용하여 최소화되는것을 구한다. w와 b는 업데이트 된다.
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),features,labels))) #100번에 힌번씩 값을 출력한다.
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc)) # test_acc의 값을 출력한다.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Iter: 0, Loss: 0.6874
Iter: 100, Loss: 0.5776
Iter: 200, Loss: 0.5349
Iter: 300, Loss: 0.5054
Iter: 400, Loss: 0.4838
Iter: 500, Loss: 0.4671
Iter: 600, Loss: 0.4535
Iter: 700, Loss: 0.4420
Iter: 800, Loss: 0.4319
Iter: 900, Loss: 0.4228
Iter: 1000, Loss: 0.4144
Testset Accuracy: 1.0000
